In [1]:
import numpy as np

In [27]:
#features:
'''
acousticness,  danceability, duration_ms*, energy, instrumentalness, key*, liveness, loudness, mode, speechiness, tempo*,  
time_signature*, valence
'''
#https://developer.spotify.com/documentation/web-api/reference/#endpoint-get-audio-features
def residual_square_sum(song1, song2):
    list_of_features = [scaled_squared_difference(song1.acousticness, song2.acousticness),
                       scaled_squared_difference(song1.danceability, song2.danceability),
                       scaled_squared_difference(song1.duration_ms, song2.duration_ms),
                       scaled_squared_difference(song1.energy, song2.energy),
                       scaled_squared_difference(song1.instrumentalness, song2.instrumentalness),
                       scaled_squared_difference_circular(song1.key, song2.key, range(12)),
                       scaled_squared_difference(song1.liveness, song2.liveness),
                       scaled_squared_difference(song1.mode, song2.mode),
                       scaled_squared_difference(song1.speechiness, song2.speechiness),
                       scaled_squared_difference(song1.tempo, song2.tempo),
                       scaled_squared_difference(song1.time_signature, song2.time_signature),
                       scaled_squared_difference(song1.valence, song2.valence)]
    return np.sum(list_of_features)
    

In [25]:
def scaled_squared_difference(num1, num2):
    maxnum = max(abs(num1), abs(num2))
    return (num1-num2 / maxnum) ** 2

def scaled_squared_difference_circular(num1, num2, circular_array):
    print(find_circular_diff(num1, num2, circular_array))
    return (find_circular_diff(num1, num2, circular_array) / 6) **2

1


0.027777777777777776

In [23]:
def find_circular_diff(num1, num2, circular_array):
    
    if num2 >= num1:
        p1 = num1
        p2 = num2
    else:
        p1 = num2
        p2 = num1
    
    dist1 = p2 - p1
    dist2 = p1 + len(circular_array) - p2 

    return min(dist1,dist2)


find_circular_diff(0, 11, range(12))

1

In [71]:
lst = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l"]
npar2 = np.array(lst)

npar = np.array([9, 4, 4, 3, 3, 9, 0, 4, 6, 0, -2])
n_lowest = np.argpartition(npar, 4)[:4]
print(n_lowest, npar[n_lowest])
x = n_lowest[np.argsort(npar[n_lowest])]

npar2[x]

[10  9  6  4] [-2  0  0  3]


array(['k', 'j', 'g', 'e'], dtype='<U1')